# Lab 07 (심화) — Tree-based Models: Advanced Topics

> **강의 시간:** 약 1시간  
> **전제 조건:** Lab 07 기본편 완료

---

## 학습 목표

| # | 목표 | 예상 시간 |
|---|---|---|
| 1 | CART 알고리즘 직접 구현 — 최적 분할 탐색 과정 시각화 | 15분 |
| 2 | Gradient Boosting 내부 원리 — 잔차 학습 단계별 추적 | 15분 |
| 3 | 모델 해석 가능성 — OOB Error, Permutation Importance, PDP | 15분 |
| 4 | 고급 앙상블 — VotingClassifier, StackingClassifier | 10분 |
| 5 | Exercise | 5분 |

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import itertools
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import fetch_openml, make_classification
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    VotingClassifier, StackingClassifier,
    HistGradientBoostingClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

# 한글 폰트
_fp = '/System/Library/Fonts/AppleGothic.ttf'
fm.fontManager.addfont(_fp)
plt.rcParams['font.family'] = fm.FontProperties(fname=_fp).get_name()
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style='whitegrid')

rng = np.random.default_rng(42)
print('설정 완료')

In [ ]:
# ── Tic-Tac-Toe 데이터 로드 (Lab07 기본편과 동일) ──────────────────────
try:
    ttt = fetch_openml('tic-tac-toe', version=1, as_frame=True, parser='auto')
    df  = ttt.frame.copy()
except Exception:
    cols  = ['TL','TM','TR','ML','C','MR','BL','BM','BR']
    lines = [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]
    rows, labels = [], []
    for combo in itertools.product(['x','o','b'], repeat=9):
        board = list(combo)
        x_wins = any(all(board[i]=='x' for i in ln) for ln in lines)
        rows.append(board)
        labels.append('positive' if x_wins else 'negative')
    df = pd.DataFrame(rows, columns=cols)
    df['class'] = labels
    df = df.sample(958, random_state=42).reset_index(drop=True)

feature_cols = [c for c in df.columns if c != 'class']
enc = OrdinalEncoder(categories=[['b','o','x']]*9)
X   = enc.fit_transform(df[feature_cols]).astype(np.float32)
y   = (df['class'] == 'positive').astype(int).values

X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f'데이터 준비 완료 — Train: {len(X_tr)}, Test: {len(X_te)}')
print(f'특성: {feature_cols}')

---
## Part 1. CART 알고리즘 직접 구현

### 1-1. CART란?

**CART (Classification and Regression Trees)** 는 sklearn 결정 트리의 핵심 알고리즘입니다.  
각 노드에서 **모든 특성 × 모든 임계값** 조합을 탐색해 가장 불순도를 낮추는 분할을 선택합니다.

```
for 각 특성 f:
    for 각 임계값 t:
        left  = 샘플[f ≤ t]
        right = 샘플[f > t]
        IG = Gini(부모) - 가중평균(Gini(left), Gini(right))
    최적 (f*, t*) = argmax IG
```

이 탐색 과정을 2D 합성 데이터에서 시각화해봅니다.

In [ ]:
# ── CART 핵심 함수 직접 구현 ───────────────────────────────────────────

def gini(y):
    """노드의 Gini 불순도"""
    if len(y) == 0:
        return 0.0
    p = np.bincount(y, minlength=2) / len(y)
    return 1.0 - (p**2).sum()

def information_gain(y, y_left, y_right):
    """분할 후 Gini 정보 이득"""
    n, nl, nr = len(y), len(y_left), len(y_right)
    return gini(y) - (nl/n)*gini(y_left) - (nr/n)*gini(y_right)

def find_best_split(X, y):
    """모든 특성 × 임계값 탐색 → 최적 분할 반환"""
    best = dict(ig=-1, feature=None, threshold=None)
    n_feat = X.shape[1]
    for f in range(n_feat):
        thresholds = np.unique(X[:, f])
        for t in thresholds[:-1]:            # 마지막 값 제외
            left_mask  = X[:, f] <= t
            right_mask = ~left_mask
            ig = information_gain(y, y[left_mask], y[right_mask])
            if ig > best['ig']:
                best = dict(ig=ig, feature=f, threshold=t)
    return best

# 2D 합성 데이터로 시각화
np.random.seed(7)
X2d = np.random.randn(200, 2).astype(np.float32)
y2d = ((X2d[:, 0]**2 + X2d[:, 1]**2) < 1.2).astype(int)  # 원형 경계

# 분할 탐색 전체 IG 지형 계산 (특성 0, 특성 1 각각)
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 특성 0과 1 각각에 대해 IG 곡선
for feat, color, ax in zip([0, 1], ['steelblue', 'tomato'], axes[:2]):
    thresholds = np.sort(np.unique(X2d[:, feat]))[:-1]
    igs = []
    for t in thresholds:
        lm = X2d[:, feat] <= t
        igs.append(information_gain(y2d, y2d[lm], y2d[~lm]))
    best_t = thresholds[np.argmax(igs)]
    ax.plot(thresholds, igs, color=color, lw=2)
    ax.axvline(best_t, color='black', linestyle='--', lw=2,
               label=f'최적 임계값 = {best_t:.2f}\nIG = {max(igs):.4f}')
    ax.set_title(f'특성 {feat} 임계값별 Information Gain')
    ax.set_xlabel(f'임계값 (x{feat})')
    ax.set_ylabel('Information Gain')
    ax.legend(fontsize=9)

# 최적 분할 시각화
best = find_best_split(X2d, y2d)
f, t = best['feature'], best['threshold']
axes[2].scatter(X2d[y2d==0, 0], X2d[y2d==0, 1],
                color='tomato', s=30, alpha=0.7, label='클래스 0')
axes[2].scatter(X2d[y2d==1, 0], X2d[y2d==1, 1],
                color='steelblue', s=30, alpha=0.7, label='클래스 1')
if f == 0:
    axes[2].axvline(t, color='black', lw=2.5, label=f'최적 분할: x0 ≤ {t:.2f}')
else:
    axes[2].axhline(t, color='black', lw=2.5, label=f'최적 분할: x1 ≤ {t:.2f}')
axes[2].set_title(f'루트 노드 최적 분할 (IG={best["ig"]:.4f})')
axes[2].legend(fontsize=9)
plt.tight_layout()
plt.show()

print(f'루트 노드 최적 분할: 특성 {f}, 임계값 {t:.4f}, IG={best["ig"]:.4f}')

### 1-2. 트리 성장 과정 — 깊이별 결정 경계 형성

In [ ]:
# 깊이 1~5에서 결정 경계가 만들어지는 과정 시각화
x0_min, x0_max = X2d[:, 0].min()-0.3, X2d[:, 0].max()+0.3
x1_min, x1_max = X2d[:, 1].min()-0.3, X2d[:, 1].max()+0.3
xx, yy = np.meshgrid(np.linspace(x0_min, x0_max, 200),
                      np.linspace(x1_min, x1_max, 200))
grid = np.c_[xx.ravel(), yy.ravel()].astype(np.float32)

fig, axes = plt.subplots(1, 5, figsize=(18, 3.5))
depths_show = [1, 2, 3, 5, 10]

for ax, d in zip(axes, depths_show):
    dt = DecisionTreeClassifier(max_depth=d, random_state=42)
    dt.fit(X2d, y2d)
    zz = dt.predict(grid).reshape(xx.shape)
    ax.contourf(xx, yy, zz, alpha=0.3, cmap='RdBu', levels=[-0.5, 0.5, 1.5])
    ax.contour(xx, yy, zz, levels=[0.5], colors='black', linewidths=1.5)
    ax.scatter(X2d[y2d==0, 0], X2d[y2d==0, 1], c='tomato',    s=18, alpha=0.8)
    ax.scatter(X2d[y2d==1, 0], X2d[y2d==1, 1], c='steelblue', s=18, alpha=0.8)
    acc = dt.score(X2d, y2d)
    leaves = dt.get_n_leaves()
    ax.set_title(f'depth={d}\nAcc={acc:.2f}, 리프={leaves}', fontsize=9)
    ax.set_xlim(x0_min, x0_max)
    ax.set_ylim(x1_min, x1_max)
    ax.axis('off')

plt.suptitle('트리 성장 과정 — 결정 경계 형성 (깊이 1→10)', y=1.02, fontsize=12)
plt.tight_layout()
plt.show()

print('depth=1:  단순 선형 경계 (고편향)')
print('depth=3:  실제 원형 패턴에 근접')
print('depth=10: 훈련 데이터 암기 시작 (고분산)')

### 1-3. Cost-Complexity Pruning (CCP)

sklearn의 **사후 가지치기(post-pruning)** 방법입니다.  
완전히 성장한 트리에서 가지를 **하나씩 제거**하면서 최적 크기를 탐색합니다.

$$\text{비용} = \text{오분류율} + \alpha \times \text{리프 수}$$

- $\alpha = 0$: 가지치기 없음 (원래 트리)
- $\alpha$ 증가 → 더 많이 가지치기 → 더 단순한 트리

In [ ]:
# Minimal Cost-Complexity Pruning
dt_full_ccp = DecisionTreeClassifier(random_state=42)
path = dt_full_ccp.cost_complexity_pruning_path(X_tr, y_tr)
alphas, impurities = path.ccp_alphas, path.impurities

# 각 alpha에 대한 train/test 정확도
tr_accs_ccp, te_accs_ccp, n_leaves_ccp = [], [], []
for a in alphas:
    dt = DecisionTreeClassifier(ccp_alpha=a, random_state=42)
    dt.fit(X_tr, y_tr)
    tr_accs_ccp.append(dt.score(X_tr, y_tr))
    te_accs_ccp.append(dt.score(X_te, y_te))
    n_leaves_ccp.append(dt.get_n_leaves())

best_alpha_idx = np.argmax(te_accs_ccp)
best_alpha     = alphas[best_alpha_idx]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(alphas, impurities, 'o-', color='steelblue', lw=2, markersize=4)
axes[0].set_title('α에 따른 트리 불순도')
axes[0].set_xlabel('ccp_alpha (α)')
axes[0].set_ylabel('전체 불순도')

axes[1].plot(alphas, n_leaves_ccp, 'o-', color='seagreen', lw=2, markersize=4)
axes[1].axvline(best_alpha, color='tomato', linestyle='--', lw=2,
                label=f'최적 α={best_alpha:.4f}')
axes[1].set_title('α에 따른 리프 수 (트리 크기)')
axes[1].set_xlabel('ccp_alpha (α)')
axes[1].set_ylabel('리프 수')
axes[1].legend()

axes[2].plot(alphas, tr_accs_ccp, 'o-', color='steelblue', lw=2,
             markersize=4, label='Train Acc')
axes[2].plot(alphas, te_accs_ccp, 's-', color='tomato',    lw=2,
             markersize=4, label='Test Acc')
axes[2].axvline(best_alpha, color='seagreen', linestyle='--', lw=2,
                label=f'최적 α={best_alpha:.4f} (Test={te_accs_ccp[best_alpha_idx]:.3f})')
axes[2].set_title('α에 따른 Train / Test 정확도')
axes[2].set_xlabel('ccp_alpha (α)')
axes[2].set_ylabel('Accuracy')
axes[2].legend(fontsize=8)

plt.tight_layout()
plt.show()

print(f'최적 ccp_alpha: {best_alpha:.6f}')
print(f'  리프 수: {n_leaves_ccp[best_alpha_idx]}')
print(f'  Test Accuracy: {te_accs_ccp[best_alpha_idx]:.4f}')

---
## Part 2. Gradient Boosting 내부 원리 심화

### 2-1. 잔차(Residual) 학습

Gradient Boosting은 이전 앙상블의 **오차(잔차)를 다음 트리가 학습**합니다.

$$F_m(x) = F_{m-1}(x) + \eta \cdot h_m(x)$$

- $F_{m-1}$: 이전까지의 앙상블 예측
- $h_m$: 잔차 $r_m = y - F_{m-1}(x)$ 를 학습한 새 트리  
- $\eta$: 학습률 (shrinkage)

회귀 문제로 단계별 잔차 감소를 먼저 시각화합니다.

In [ ]:
# ── 회귀 문제로 Gradient Boosting 잔차 학습 시각화 ──────────────────────
np.random.seed(0)
x_reg  = np.linspace(0, 6, 150)
y_reg  = np.sin(x_reg) + 0.3*np.random.randn(150)
X_reg  = x_reg.reshape(-1, 1)
x_eval = np.linspace(0, 6, 300).reshape(-1, 1)

# 단계별 Gradient Boosting 수동 구현
STAGES = [1, 2, 3, 5, 10, 50]
LR     = 0.3
F      = np.full(len(x_reg), y_reg.mean())   # F_0 = 평균
F_eval = np.full(300, y_reg.mean())
trees  = []

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
stage_idx = {s: i for i, s in enumerate(STAGES)}

for m in range(1, 51):
    residuals = y_reg - F
    h = DecisionTreeRegressor(max_depth=2, random_state=m)
    h.fit(X_reg, residuals)
    trees.append(h)
    F      += LR * h.predict(X_reg)
    F_eval += LR * h.predict(x_eval)

    if m in stage_idx:
        ax  = axes[stage_idx[m] // 3][stage_idx[m] % 3]
        res = y_reg - (F - LR * h.predict(X_reg))  # m번째 직전 잔차
        ax.scatter(x_reg, y_reg, s=8, color='gray', alpha=0.4, label='데이터')
        ax.plot(x_eval.ravel(), F_eval, color='tomato', lw=2.5, label=f'F_{m} (앙상블)')
        ax.plot(x_eval.ravel(), np.sin(x_eval.ravel()), 'k--',
                lw=1.5, alpha=0.5, label='실제 함수')
        ax2 = ax.twinx()
        ax2.bar(x_reg, res, width=0.04, alpha=0.25, color='steelblue')
        ax2.set_ylabel('잔차', color='steelblue', fontsize=8)
        ax2.tick_params(axis='y', labelcolor='steelblue', labelsize=7)
        rmse = np.sqrt(np.mean((y_reg - F)**2))
        ax.set_title(f'Stage {m} — RMSE={rmse:.4f}', fontsize=10)
        ax.legend(fontsize=7, loc='upper right')

plt.suptitle('Gradient Boosting 단계별 잔차 감소\n(파란 막대=현재 잔차, 빨간선=앙상블 예측)', y=1.02)
plt.tight_layout()
plt.show()

### 2-2. 학습률 × 트리 수 트레이드오프

Gradient Boosting의 핵심 하이퍼파라미터:

| 파라미터 | 역할 | 증가 시 |
|---|---|---|
| `n_estimators` | 트리 수 | 성능↑, 과적합 위험↑, 속도↓ |
| `learning_rate` ($\eta$) | 각 트리 기여도 축소 | 낮을수록 안정적이지만 더 많은 트리 필요 |
| `max_depth` | 개별 트리 깊이 | 얕을수록(2~5) 좋음 |

> **경험 법칙:** `learning_rate`를 낮추면 `n_estimators`를 비례해서 늘려야 합니다.

In [ ]:
# 학습률 × 트리 수 조합별 성능
lr_list = [0.3, 0.1, 0.05]
n_list  = list(range(1, 201, 5))
colors_lr = ['tomato', 'steelblue', 'seagreen']

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

for lr, color in zip(lr_list, colors_lr):
    tr_accs_gb, te_accs_gb = [], []
    for n in n_list:
        gb = GradientBoostingClassifier(
            n_estimators=n, learning_rate=lr,
            max_depth=3, random_state=42
        )
        gb.fit(X_tr, y_tr)
        tr_accs_gb.append(gb.score(X_tr, y_tr))
        te_accs_gb.append(gb.score(X_te, y_te))
    axes[0].plot(n_list, tr_accs_gb, color=color, lw=1.5, linestyle='--', alpha=0.6)
    axes[1].plot(n_list, te_accs_gb, color=color, lw=2,
                 label=f'lr={lr}')

for ax, title in zip(axes, ['Train Accuracy', 'Test Accuracy']):
    ax.set_title(title + ' — 학습률별 수렴 비교')
    ax.set_xlabel('n_estimators')
    ax.set_ylabel('Accuracy')
    ax.set_ylim(0.7, 1.05)
axes[1].legend()
plt.tight_layout()
plt.show()

print('낮은 학습률(0.05): 느리게 수렴하지만 더 안정적')
print('높은 학습률(0.3) : 빠르게 수렴하지만 과적합 위험')

### 2-3. HistGradientBoosting — 현대적 고속 구현

sklearn의 `HistGradientBoostingClassifier`는 **LightGBM** 방식을 채택합니다:

| | GradientBoosting | **HistGradientBoosting** |
|---|---|---|
| 분할 탐색 | 모든 값 확인 $O(nd)$ | 히스토그램 빈 $O(kd)$ |
| NaN 처리 | 수동 전처리 필요 | **내장 지원** |
| 샘플 수 | 소규모 적합 | **대규모 적합** |
| 조기 종료 | 없음 | **내장 지원** |

In [ ]:
import time

# GB vs HistGB 속도/성능 비교
configs = [
    ('GradientBoosting\n(n=200)',
     GradientBoostingClassifier(n_estimators=200, learning_rate=0.1,
                                max_depth=3, random_state=42)),
    ('HistGradientBoosting\n(n=200)',
     HistGradientBoostingClassifier(max_iter=200, learning_rate=0.1,
                                    max_depth=3, random_state=42)),
    ('HistGradientBoosting\n(조기 종료)',
     HistGradientBoostingClassifier(max_iter=500, learning_rate=0.05,
                                    max_depth=3, random_state=42,
                                    early_stopping=True, n_iter_no_change=20,
                                    validation_fraction=0.15)),
]

print(f'{"모델":<35} {"Test Acc":>10} {"학습시간(s)":>12}')
print('-' * 60)
for name, model in configs:
    t0 = time.time()
    model.fit(X_tr, y_tr)
    elapsed = time.time() - t0
    te_acc  = model.score(X_te, y_te)
    n_iters = getattr(model, 'n_iter_', getattr(model, 'n_estimators', '?'))
    print(f'{name.replace(chr(10)," "):<35} {te_acc:>10.4f} {elapsed:>12.3f}s  (iter={n_iters})')

# HistGB 조기 종료 학습 곡선
hgb_es = configs[2][1]
if hasattr(hgb_es, 'train_score_'):
    fig, ax = plt.subplots(figsize=(8, 3))
    ax.plot(hgb_es.train_score_,    color='steelblue', lw=2, label='Train')
    ax.plot(hgb_es.validation_score_, color='tomato', lw=2, label='Validation')
    ax.axvline(hgb_es.n_iter_-1, color='seagreen', linestyle='--', lw=2,
               label=f'조기 종료 iter={hgb_es.n_iter_}')
    ax.set_title('HistGradientBoosting 조기 종료 학습 곡선')
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Score')
    ax.legend()
    plt.tight_layout()
    plt.show()

---
## Part 3. 모델 해석 가능성 (Interpretability)

### 3-1. OOB Error — 공짜 교차 검증

랜덤 포레스트의 각 트리는 **부트스트랩 샘플**로 학습합니다.  
각 샘플이 학습에 포함되지 않은 트리들만으로 예측 → **OOB(Out-of-Bag) Error**

```
전체 데이터: [1, 2, 3, 4, 5, 6]
트리1 학습샘플: [1, 3, 3, 5, 6]  → OOB 샘플: [2, 4]
트리2 학습샘플: [2, 2, 4, 5, 6]  → OOB 샘플: [1, 3]
...
각 샘플은 평균 ~37%의 트리에서 OOB
```

별도 검증 셋 없이 **과적합 여부를 무료로 추정**할 수 있습니다.

In [ ]:
# OOB Error vs Cross-Validation 비교
n_list_oob = [5, 10, 20, 50, 100, 200]
oob_scores, cv5_scores, te_scores = [], [], []

for n in n_list_oob:
    rf_oob = RandomForestClassifier(
        n_estimators=n, oob_score=True, random_state=42, n_jobs=-1
    )
    rf_oob.fit(X_tr, y_tr)
    oob_scores.append(rf_oob.oob_score_)
    te_scores.append(rf_oob.score(X_te, y_te))
    cv5_scores.append(
        cross_val_score(
            RandomForestClassifier(n_estimators=n, random_state=42, n_jobs=-1),
            X, y, cv=5
        ).mean()
    )

fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(n_list_oob, oob_scores, 'o-', color='steelblue', lw=2, label='OOB Score')
ax.plot(n_list_oob, cv5_scores, 's-', color='tomato',    lw=2, label='5-Fold CV Score')
ax.plot(n_list_oob, te_scores,  'd--',color='seagreen',  lw=2, label='Test Score (실제)')
ax.set_title('OOB Score vs Cross-Validation Score\n(OOB는 공짜 교차 검증)')
ax.set_xlabel('n_estimators')
ax.set_ylabel('Accuracy')
ax.set_ylim(0.8, 1.01)
ax.legend()
plt.tight_layout()
plt.show()

rf200 = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42, n_jobs=-1)
rf200.fit(X_tr, y_tr)
print(f'n=200 기준 | OOB={rf200.oob_score_:.4f} | CV-5={cv5_scores[-1]:.4f} | Test={te_scores[-1]:.4f}')
print('OOB Score ≈ CV Score → 별도 검증 셋 없이도 신뢰할 수 있는 성능 추정 가능')

### 3-2. MDI vs Permutation Feature Importance

기본편의 특성 중요도(MDI)는 **편향 문제**가 있습니다:

| 방법 | 계산 방법 | 편향 |
|---|---|---|
| **MDI** (Mean Decrease Impurity) | 트리의 불순도 감소량 평균 | 고유값 많은 연속형 특성에 유리 |
| **Permutation Importance** | 특성을 랜덤 섞었을 때 성능 감소량 | **편향 없음**, 느림 |

Tic-Tac-Toe처럼 범주형 특성에서도 두 방법의 결과가 다를 수 있습니다.

In [ ]:
# MDI vs Permutation Importance 비교
rf_imp = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
rf_imp.fit(X_tr, y_tr)

# MDI Importance
mdi_imp = rf_imp.feature_importances_

# Permutation Importance (Test 셋 기준)
perm_result = permutation_importance(
    rf_imp, X_te, y_te,
    n_repeats=30, random_state=42, n_jobs=-1
)
perm_imp   = perm_result.importances_mean
perm_std   = perm_result.importances_std

# 정렬 (MDI 기준)
sorted_idx = np.argsort(mdi_imp)[::-1]
feat_names_sorted = [feature_cols[i] for i in sorted_idx]

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# MDI
bars = axes[0].bar(feat_names_sorted, mdi_imp[sorted_idx],
                   color='steelblue', edgecolor='k', alpha=0.85)
axes[0].set_title('MDI Importance (트리 내 불순도 감소)')
axes[0].set_xlabel('보드 위치')
axes[0].set_ylabel('Importance')
for bar, v in zip(bars, mdi_imp[sorted_idx]):
    axes[0].text(bar.get_x()+bar.get_width()/2, v+0.002,
                 f'{v:.3f}', ha='center', fontsize=7.5)

# Permutation (MDI와 같은 순서로)
pm_sorted = perm_imp[sorted_idx]
ps_sorted = perm_std[sorted_idx]
bars2 = axes[1].bar(feat_names_sorted, pm_sorted,
                    yerr=ps_sorted, color='tomato', edgecolor='k',
                    alpha=0.85, capsize=4)
axes[1].set_title('Permutation Importance (성능 감소량, n_repeats=30)')
axes[1].set_xlabel('보드 위치')
axes[1].set_ylabel('Mean Accuracy Decrease')
for bar, v in zip(bars2, pm_sorted):
    axes[1].text(bar.get_x()+bar.get_width()/2, v + ps_sorted[list(pm_sorted).index(v)] + 0.002,
                 f'{v:.3f}', ha='center', fontsize=7.5)

plt.tight_layout()
plt.show()

# 순위 상관계수
from scipy.stats import spearmanr
corr, pval = spearmanr(mdi_imp, perm_imp)
print(f'MDI vs Permutation Importance 순위 상관계수: {corr:.4f} (p={pval:.4f})')
print('→ 두 방법의 순위가 다른 특성은 MDI 편향의 영향을 받은 것')

### 3-3. Partial Dependence Plot (PDP)

**PDP**는 특정 특성이 변할 때 모델 예측이 어떻게 변하는지를 보여줍니다.  
다른 특성들은 **평균화(marginalize)** 하여 단일 특성의 순수 효과를 추출합니다.

$$\text{PDP}(x_j) = \mathbb{E}_{X_{-j}}\left[F(x_j, X_{-j})\right] \approx \frac{1}{n}\sum_{i=1}^{n} F(x_j, x_{-j}^{(i)})$$

2D PDP는 **두 특성의 상호작용 효과**를 시각화합니다.

In [ ]:
# Partial Dependence Plot (1D × 4개 + 2D × 1개)
# 중요도 상위 4개 특성
top4_idx = sorted_idx[:4]
top4_names = feat_names_sorted[:4]

# 1D PDP (상위 4개)
fig, axes = plt.subplots(1, 4, figsize=(16, 3.5))
PartialDependenceDisplay.from_estimator(
    rf_imp, X_tr, features=list(top4_idx),
    feature_names=feature_cols,
    kind='both',        # 'average' + 개별 ICE 곡선
    ax=axes,
    ice_lines_kw=dict(color='steelblue', alpha=0.05, lw=0.5),
    pd_line_kw=dict(color='tomato', lw=2.5),
    subsample=200,
    random_state=42,
)
for ax, name in zip(axes, top4_names):
    ax.set_title(f'PDP — {name}')
    ax.set_xlabel('인코딩값 (b=0, o=1, x=2)')
    ax.set_ylabel('X승리 확률')
plt.suptitle('1D Partial Dependence Plot (빨간선=평균효과, 파란선=개별ICE)', y=1.04)
plt.tight_layout()
plt.show()

# 2D PDP (상위 2개 특성 상호작용)
f0, f1 = int(top4_idx[0]), int(top4_idx[1])
fig, ax = plt.subplots(figsize=(6, 5))
PartialDependenceDisplay.from_estimator(
    rf_imp, X_tr, features=[(f0, f1)],
    feature_names=feature_cols,
    kind='average',
    ax=ax,
)
ax.set_title(f'2D PDP — {feature_cols[f0]} × {feature_cols[f1]}\n(두 특성의 상호작용 효과)')
plt.tight_layout()
plt.show()

print('PDP 해석:')
print('  x=2(X칸)일 때 확률 급상승 → X가 해당 칸을 차지하는 게 승리에 결정적')
print('  o=1(O칸)일 때 확률 하락  → 상대가 차지하면 불리')

---
## Part 4. 고급 앙상블 전략

### 4-1. VotingClassifier — 소프트 투표 vs 하드 투표

서로 다른 종류의 모델을 결합합니다:

```
하드 투표: 각 모델 예측 클래스의 다수결
   모델1: 클래스1  모델2: 클래스1  모델3: 클래스0  →  클래스1 (2:1)

소프트 투표: 각 모델의 확률 평균
   모델1: [0.7, 0.3]  모델2: [0.6, 0.4]  모델3: [0.4, 0.6]
   → 평균: [0.57, 0.43]  →  클래스0
```

In [ ]:
# 개별 기반 모델
dt_base = DecisionTreeClassifier(max_depth=5, random_state=42)
rf_base = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
gb_base = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,
                                      max_depth=3, random_state=42)

# 하드 투표
hard_voter = VotingClassifier(
    estimators=[('dt', dt_base), ('rf', rf_base), ('gb', gb_base)],
    voting='hard'
)

# 소프트 투표 (확률 기반)
soft_voter = VotingClassifier(
    estimators=[('dt', dt_base), ('rf', rf_base), ('gb', gb_base)],
    voting='soft'
)

configs_vote = [
    ('DecisionTree (단독)', dt_base),
    ('RandomForest (단독)', rf_base),
    ('GradientBoosting (단독)', gb_base),
    ('VotingClassifier (Hard)', hard_voter),
    ('VotingClassifier (Soft)', soft_voter),
]

print(f'{"모델":<32} {"Train":>8} {"Test":>8} {"CV-5":>8}')
print('-' * 60)
vote_results = {}
for name, model in configs_vote:
    model.fit(X_tr, y_tr)
    tr = model.score(X_tr, y_tr)
    te = model.score(X_te, y_te)
    cv = cross_val_score(model, X, y, cv=5).mean()
    vote_results[name] = dict(train=tr, test=te, cv=cv)
    print(f'{name:<32} {tr:>8.4f} {te:>8.4f} {cv:>8.4f}')

### 4-2. StackingClassifier — 메타 러너

**스태킹**은 기반 모델의 예측을 **새로운 특성으로 사용해** 메타 모델을 학습합니다.

```
Level-0 (기반 모델)
  ├─ DecisionTree  → 예측 확률 p1
  ├─ RandomForest  → 예측 확률 p2   → [p1, p2, p3] → LogReg (메타) → 최종 예측
  └─ GradBoost     → 예측 확률 p3

Level-1 (메타 모델)
  └─ LogisticRegression (기반 모델의 예측을 입력으로)
```

In [ ]:
# StackingClassifier
stacking = StackingClassifier(
    estimators=[
        ('dt', DecisionTreeClassifier(max_depth=5, random_state=42)),
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)),
        ('gb', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,
                                          max_depth=3, random_state=42)),
    ],
    final_estimator=LogisticRegression(max_iter=1000, random_state=42),
    cv=5,
    passthrough=False,
)
stacking.fit(X_tr, y_tr)
st_tr = stacking.score(X_tr, y_tr)
st_te = stacking.score(X_te, y_te)
st_cv = cross_val_score(stacking, X, y, cv=5).mean()
print(f'StackingClassifier     Train={st_tr:.4f}  Test={st_te:.4f}  CV-5={st_cv:.4f}')

# 메타 모델 가중치 확인
meta = stacking.final_estimator_
print(f'\n메타 모델(LogReg) 가중치: DT={meta.coef_[0][0]:.3f}  RF={meta.coef_[0][1]:.3f}  GB={meta.coef_[0][2]:.3f}')
print('→ 가중치가 클수록 해당 기반 모델에 더 많이 의존')

# 최종 비교 그래프
all_results = {
    'DT': vote_results['DecisionTree (단독)'],
    'RF': vote_results['RandomForest (단독)'],
    'GB': vote_results['GradientBoosting (단독)'],
    'Hard Vote': vote_results['VotingClassifier (Hard)'],
    'Soft Vote': vote_results['VotingClassifier (Soft)'],
    'Stacking': dict(train=st_tr, test=st_te, cv=st_cv),
}

x_pos = np.arange(len(all_results))
width = 0.28
fig, ax = plt.subplots(figsize=(13, 4))
for i, (key, label, color) in enumerate([
    ('train', 'Train', 'steelblue'),
    ('test',  'Test',  'tomato'),
    ('cv',    'CV-5',  'seagreen')
]):
    vals = [v[key] for v in all_results.values()]
    bars = ax.bar(x_pos + (i-1)*width, vals, width,
                  label=label, color=color, edgecolor='k', alpha=0.85)
    for bar, v in zip(bars, vals):
        ax.text(bar.get_x()+bar.get_width()/2, v+0.004,
                f'{v:.2f}', ha='center', fontsize=7)

ax.set_title('앙상블 전략 최종 비교')
ax.set_xticks(x_pos)
ax.set_xticklabels(list(all_results.keys()))
ax.set_ylabel('Accuracy')
ax.set_ylim(0.7, 1.1)
ax.legend()
plt.tight_layout()
plt.show()

---
## Exercise

### Exercise 1. CCP 가지치기 vs max_depth 가지치기 비교

같은 Tic-Tac-Toe 데이터에서:
1. `max_depth` 탐색으로 최적 Test Accuracy 달성하는 깊이 $d^*$를 찾으세요.
2. `ccp_alpha` 탐색으로 최적 Test Accuracy 달성하는 $\alpha^*$를 찾으세요.
3. 두 방법의 **리프 수**, **Test Accuracy**, **Train Accuracy**를 비교하세요.  
   어느 방법이 더 간결하면서 정확한 트리를 만드나요?

In [ ]:
# Exercise 1: CCP vs max_depth 가지치기 비교
# Your code here

### Exercise 2. 랜덤 포레스트 특성 제거 실험

**재귀적 특성 제거(RFE, Recursive Feature Elimination)** 방식으로:  
1. Permutation Importance가 가장 낮은 특성부터 하나씩 제거합니다.
2. 매 제거 후 5-Fold CV Accuracy를 측정합니다.
3. **몇 개의 특성만으로도 전체 특성과 유사한 성능을 낼 수 있는지** 그래프로 확인하세요.

힌트: `permutation_importance` 결과를 활용하고, 특성 수 9→8→7→...→1 순서로 반복하세요.

In [ ]:
# Exercise 2: 재귀적 특성 제거 실험
# Your code here

### Exercise 3. (도전) 커스텀 앙상블 설계

다음 조건을 만족하는 앙상블을 설계하세요:

- 기반 모델 3개 이상 (자유 선택, 하이퍼파라미터 직접 튜닝)
- **메타 모델**: `RandomForestClassifier` (LogisticRegression 대신)
- `passthrough=True` 옵션 사용 (기반 모델 예측 + 원본 특성을 메타에 함께 전달)
- 5-Fold CV 기준 **0.97 이상** 달성 목표

최종 모델의 성능과 메타 모델이 어떤 입력을 중요하게 보는지 분석하세요.

In [ ]:
# Exercise 3: 커스텀 스태킹 앙상블
# Your code here

---
## Summary

| 개념 | 핵심 내용 |
|---|---|
| **CART 알고리즘** | 모든 특성×임계값 탐색 → 최대 IG 분할 선택 |
| **CCP Pruning** | $\text{비용} = \text{오분류율} + \alpha \cdot \text{리프수}$ — 사후 가지치기 |
| **GB 잔차 학습** | $F_m = F_{m-1} + \eta \cdot h_m(\text{잔차})$ — 단계별 오차 보정 |
| **HistGB** | 히스토그램 기반 빠른 GB — 대규모·NaN·조기종료 지원 |
| **OOB Error** | 부트스트랩 비포함 샘플로 무료 검증 ≈ CV |
| **MDI vs Perm** | MDI=편향있음, Permutation=편향없음(느림) |
| **PDP** | 단일 특성의 순수 효과 시각화 / 2D=상호작용 효과 |
| **Hard Voting** | 다수결 — 각 모델의 클래스 예측 |
| **Soft Voting** | 확률 평균 — 더 정교한 결합 |
| **Stacking** | 기반 모델 예측을 새 특성으로 → 메타 모델 학습 |

---

**다음 강의 (Week 8):** 신경망 — 다층 퍼셉트론, 활성화 함수, 역전파 알고리즘